# Stochastic Models and Simulations of Phototaxis
[Link](https://www.researchgate.net/profile/Devaki-Bhaya/publication/265289716_Stochastic_Models_and_Simulations_of_Phototaxis/links/5535445a0cf268fd0015d694/Stochastic-Models-and-Simulations-of-Phototaxis.pdf)

In [27]:
using Random, LinearAlgebra, Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


# Functions

In [96]:
#=
Sample N points from a circle of radius R
https://stackoverflow.com/questions/5837572/generate-a-random-point-within-a-circle-uniformly/50746409
=#
function sampleCircle(N, R)
    # Sample radii
    # PDF = 2x; CDF = x²; CDF⁻¹ = √x
    radii = R * sqrt.(rand(N))
    
    # Sample angles
    # theta = 0 - 2π
    thetas = rand(N) * 2π

    # points
    X = @.radii * [cos(thetas) sin(thetas)]
    return X
end

#=
Compute all pairwise distances between each point in X
=#
function pairwiseDistance(X)
    n = size(X)[1]
    dists = zeros(n, n)
    for i in 1:n
        a = X[i, :]
        for j in i:n
            b = X[j, :]
            # Compute distance
            d = norm(a - b)
            # Save
            dists[i, j] = dists[j, i] = d
        end
    end
end

pairwiseDistance (generic function with 1 method)

# Local interactions
At any
given point in time, we assume that every cell may move according to one of the
following three options (shown in Figure 2):
1. a cell may continue to move without changing its previous direction
2.  a cell can stop moving
3. a cell may orient itself. In this case it will moves in the direction of one of its
neighboring cells. The candidate neighboring cells must be within a certain
interaction distance (shown as the dotted circle in Figure 2(c)).

In [ ]:
function localInteractions(θ, v, X, p)
    θ = 
end

In [97]:
"""Parameters"""
N = 250
R = 200
D = 5

"""Initialize"""
# Initialize points
X = sampleCircle(N, R)
# Pairwise distances
dists = pairwiseDistance(X)
# θ ∈ {0, 2π}
θ = rand(N) * 2π
# v ∈ {0, 1}
v = rand(N) .> .5


250-element BitArray{1}:
 0
 1
 0
 0
 0
 0
 0
 0
 1
 0
 1
 1
 1
 ⋮
 1
 1
 0
 1
 1
 1
 0
 1
 0
 0
 0
 1

In [46]:
N = 200
R = 10
X = sampleCircle(N, R)

200×2 Array{Float64,2}:
 -1.77923    -2.55243
 -7.42249     1.9623
 -5.60328    -3.41909
 -8.71454     4.71021
  5.80142     5.97382
  2.31748    -8.43708
  6.39915    -7.48753
  6.77921    -1.32499
 -0.686979    6.85008
 -0.678875   -2.6047
  5.68672     5.33583
  4.67874    -5.90765
 -3.15831     9.15059
  ⋮          
  0.0580153   9.43056
 -1.60137     9.64862
 -1.87887    -6.39934
 -8.92327    -1.48463
 -3.69557    -2.32217
  0.235187    9.02702
  0.157357   -2.36029
  7.33592    -5.16345
  6.88368    -4.56813
 -1.96946    -1.79401
 -5.68487    -3.07052
 -6.35581     6.98411

102.69462221554419

In [94]:
size(X)[1]

200